In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import re
import fnmatch
import matplotlib.pyplot as plt
from functools import reduce
from bs4 import BeautifulSoup

# Import planning area

In [ ]:

planningArea_noSea = gpd.read_file(r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_Masterplan\MasterPlan2019PlanningAreaBoundaryNoSea.geojson")

def extract_html_tags(row):
    html_content = row["Description"]
    soup = BeautifulSoup(html_content,"html.parser")
    ths = soup.find_all('th')
    tds = soup.find_all('td')
    column_names = []
    column_values = []
    for th in ths:
        column_names.append(th.get_text())
    for td in tds:
        column_values.append(td.get_text())
    column_names = column_names[1:] # remove the first "Attribute"
    return pd.Series({k:v for k,v in zip(column_names,column_values)})
# column_names, _ = extract_html_tags(planningArea_noSea.iloc[0,1])
planningArea_shp = planningArea_noSea.join(planningArea_noSea.apply(extract_html_tags,axis=1))
print(type(planningArea_shp))
planningArea_shp.plot(fc='white',ec='k')
planningArea_shp.head()

# Data analysis of property prices

In [14]:
residential_transaction_dir = r"Data\Residential Transaction"
residential_transaction_fps = [os.path.join(residential_transaction_dir,d,fp) for d in os.listdir(residential_transaction_dir) for fp in os.listdir(os.path.join(residential_transaction_dir,d))]

# residential_transaction_dfs = [pd.read_csv(fp) for fp in residential_transaction_fps]
# residential_transaction_dfs[0].head()

In [20]:
residential_transaction_dfs = []
for fp in residential_transaction_fps:
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        df = pd.read_csv(fp, encoding='latin-1')
    residential_transaction_dfs.append(df)
residential_transaction_dfs[0]

residential_transaction_df = pd.concat(residential_transaction_dfs)
print(f"length of df: {len(residential_transaction_df)}")
residential_transaction_df.head()

length of df: 267946


,Project Name,Transacted Price ($),Area (SQFT),Unit Price ($ PSF),Sale Date,Address,Type of Sale,Type of Area,Area (SQM),Unit Price ($ PSM),...,Property Type,Number of Units,Tenure,Completion Date,Purchaser Address Indicator,Postal Code,Postal District,Postal Sector,Planning Region,Planning Area
0,CASTLE GREEN,"950,000","1,151.75",825,02 Jan 2014,481 YIO CHU KANG ROAD #05-02,Resale,Strata,107,"8,879",...,Condominium,1,99 yrs from 01/12/1993,1997,HDB,787056,26,78,North East Region,Ang Mo Kio
1,SUMMERHILL,"1,650,000","1,550.02","1,065",02 Jan 2014,52 HUME AVENUE #01-16,Resale,Strata,144,"11,458",...,Condominium,1,Freehold,2002,Private,596230,21,59,West Region,Bukit Batok
2,THOMSON THREE,"1,051,000",731.95,"1,436",02 Jan 2014,41 BRIGHT HILL DRIVE #14-01,New Sale,Strata,68,"15,456",...,Apartment,1,99 yrs from 07/11/2012,Uncompleted,Private,573893,20,57,Central Region,Bishan
3,KINGSFORD . HILLVIEW PEAK,"730,000",516.67,"1,413",03 Jan 2014,103 HILLVIEW RISE #10-11,New Sale,Strata,48,"15,208",...,Condominium,1,99 yrs from 04/06/2012,Uncompleted,Private,667982,23,66,West Region,Bukit Batok
4,MEADOWS @ PEIRCE,"2,518,000","3,035.45",830,03 Jan 2014,628 UPPER THOMSON ROAD #05-61,Resale,Strata,282,"8,929",...,Condominium,1,Freehold,2012,Private,787131,26,78,North East Region,Ang Mo Kio


In [23]:
print(f"Number of unique project name: {len(np.unique(residential_transaction_df['Project Name']))}")
print(f"Number of unique property types: {len(np.unique(residential_transaction_df['Property Type']))}")
print(f"Min of unit price PSM: {residential_transaction_df["Unit Price ($ PSM)"].min()}; Max of unit price PSM: {residential_transaction_df["Unit Price ($ PSM)"].max()}")
print(f"Number of unique planning region: {len(np.unique(residential_transaction_df["Planning Region"]))}")
print(f"Number of unique planning area: {len(np.unique(residential_transaction_df["Planning Area"]))}")

Number of unique project name: 3218
Number of unique property types: 6
Min of unit price PSM: 1,072; Max of unit price PSM: 9,998
Number of unique planning region: 5
Number of unique planning area: 42
